In [1]:
from app import create_app
from app.models import *
from app.utils import *
from datetime import datetime as dt
from dotenv import  load_dotenv
import os
from random import randint

app = create_app()

c:\Users\sebva\.virtualenvs\advice-persona-api-US8xno1K\lib\site-packages\flask_restx\api.py:322: UserWarning: The setup method 'add_url_rule' can no longer be called on the blueprint 'api'. It has already been registered at least once, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the blueprint are done before registering it.
This warning will become an exception in Flask 2.3.
  app_or_blueprint.add_url_rule(self._doc, "doc", self.render_doc)
c:\Users\sebva\.virtualenvs\advice-persona-api-US8xno1K\lib\site-packages\flask\blueprints.py:490: UserWarning: The setup method 'record' can no longer be called on the blueprint 'api'. It has already been registered at least once, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the blueprint are done before registering it.
This warning will become an exception in Flask 2.3.
  self.record(
c:\Users\sebva\.virtualen

## User

### Create admin and 10 test users

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    if not User.query.filter_by(username='admin').first():
        # add admin
        admin = User(username = 'admin', email = 'admin@example.com', password = "a234567!")
        objs_to_add.append(admin)
    else: pass
    for idx, user in enumerate(range(10)):
        random_num = randint(1000, 10000000)
        username = f'test_{random_num}'
        u = User(username = username, email = f'{username}@example.com', password = "a234567!")
        objs_to_add.append(u)
    
    ##################################################
    db.session.add_all(objs_to_add)
    add_to_db, msg = commit_to_db(db)
    print(msg)
    db.session.close()




### Create admin and user roles

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    admin = Role(name = 'admin', description = 'Administrator')
    user = Role(name = 'user', description = 'User')
    objs_to_add.extend([admin, user])
    
    
    ##################################################
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

### Assign user roles

In [3]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    user_admin = User.query.filter_by(username="admin").first()
    all_users = User.query.filter(User.username!= "admin").all()
    role_admin = Role.query.filter_by(name="admin").first()
    role_user = Role.query.filter_by(name="user").first()
    objs_to_add.append(UserRole(user=user_admin, role=role_admin))
    
    for u in all_users:
        objs_to_add.append(UserRole(user=u, role=role_user))
    
    ##################################################  
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

success


## Persona

### Create 3 Personas

In [13]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    chad, stacy, unknown = Persona(name="Chad"), Persona(name="Stacy"), Persona(name="Unknown")
    objs_to_add = [chad, stacy, unknown]
    
    ##################################################
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

success


## Advice

### Add two advice slips from "Unknown" persona

In [27]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    # Advice is a child of Entity, so first create an Entity.
    advice_entity_1 = Entity(type = "advice")
    advice_entity_2 = Entity(type = "advice")
    objs_to_add.extend([advice_entity_1, advice_entity_2])
    
    # Proceed only if the entity is saved successfully.
    persona = Persona.query.filter_by(name = "Unknown").first()
    advice1 = Advice(
        entity = advice_entity_1, 
        persona = persona,
        content = "Remember that spiders are more afraid of you, than you are of them.",
        adviceslip_id = 1)
    advice2 = Advice(
        entity = advice_entity_2, 
        persona = persona,
        content = "The more ideas that you give away, the more ideas that will come to you.",
        adviceslip_id = 54)
    objs_to_add.extend([advice1, advice2])            
    
    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

success


### User 'admin' likes the advice about spiders.

In [29]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    user = User.query.filter_by(username = 'admin').first()
    advice = Advice.query.filter_by(adviceslip_id = 1).first()
    like = EntityLike(user = user, entity = advice.entity)
    objs_to_add.append(like)
    
    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

success
